In [ ]:
# Clone repo and install dependencies
%%capture
!git clone https://github.com/shahineb/jax-esm-emulation.git
%cd jax-esm-emulation/examples
!pip install equinox diffrax healpy xarray cartopy

In [ ]:
# Import used dependencies
from draw_samples import build_emulator


# Compile and load emulator (takes ~1min)
emulator = build_emulator(config, n_samples=1, n_steps=30)

In [ ]:
ΔT = 2.0  # Temperature anomaly in °C
month = 1  # January
seed = 0  # Random seed for reproducibility
samples = emulator(ΔT=ΔT, month=month, seed=seed)

In [ ]:
varnames = ["tas", "pr", "hurs", "sfcWind"]
nlat, nlon = config.model.input_size[1], config.model.input_size[2]
ds = xr.Dataset(
    {
        var: (('member', 'lat', 'lon'), samples[:, i, :, :])
        for i, var in enumerate(config.data.variables)
    },
    coords={
        'member': jnp.arange(len(samples)) + 1,
        'lat': jnp.arange(nlat),
        'lon': jnp.arange(nlon),
    }
)

In [ ]:
# do your plots